In [272]:
# Import all libraries
import pandas as pd


# Import data from xls
og_df = pd.read_excel('GSAF5.xls')

# Check for basic shape and some values
print(og_df.shape)
print(og_df.head(5))

#Let's create a copy to work with and rename columns
df = og_df.copy()
df.columns = df.columns.str.replace(' ', '')
print(df.columns)

(6969, 23)
          Date    Year        Type    Country              State  \
0  15 Mar 2024  2024.0  Unprovoked  AUSTRALIA         Queensland   
1  04 Mar 2024  2024.0  Unprovoked        USA             Hawaii   
2  02 Mar-2024  2024.0  Unprovoked        USA             Hawaii   
3  25 Feb-2024  2024.0  Unprovoked  AUSTRALIA  Western Australia   
4  14 Feb-2024  2024.0  Unprovoked      INDIA        Maharashtra   

                           Location  Activity                 Name Sex  Age  \
0                     Bargara Beach  Swimming       Brooklyn Sauer   F   13   
1                Old Man's, Waikiki   Surfing        Matthew White   M  NaN   
2                    Rainbows, Oahu  Swimming                  NaN   F   11   
3        Sandlnd Island, Jurian Bay       NaN               female   F   46   
4  Vaitarna River, Palghar District   Fishing  Vicky Suresh Govari   M   32   

   ...        Species                      Source  pdf href formula href  \
0  ...     Tiger shark      Y

In [273]:
#this columns don't seem to hold relevant information for this project
df.drop(columns=['Unnamed:21'], inplace=True)
df.drop(columns=['Unnamed:22'], inplace=True)
df.drop(columns=['Unnamed:11'], inplace=True)


In [274]:
#Let's drop duplicates and missing values
print(df.isnull().sum())

print(df.drop_duplicates(inplace=True))

df.dropna(subset=["Date"],axis=0,inplace=True)


Date               25
Year               27
Type               43
Country            75
State             507
Location          590
Activity          611
Name              245
Sex               604
Age              3019
Injury             60
Time             3551
Species          3157
Source             44
pdf               170
hrefformula       150
href              173
CaseNumber        171
CaseNumber.1      172
originalorder     170
dtype: int64
None


In [275]:
print("Before\n", df.eq(" ").sum())
df = df[df!= " "] #Very low number of spaces so we can drop this outliers
print("After\n", df.eq(" ").sum())


Before
 Date             0
Year             0
Type             0
Country          0
State            0
Location         0
Activity         1
Name             0
Sex              0
Age              2
Injury           1
Time             2
Species          7
Source           0
pdf              0
hrefformula      0
href             0
CaseNumber       0
CaseNumber.1     0
originalorder    0
dtype: int64
After
 Date             0
Year             0
Type             0
Country          0
State            0
Location         0
Activity         0
Name             0
Sex              0
Age              0
Injury           0
Time             0
Species          0
Source           0
pdf              0
hrefformula      0
href             0
CaseNumber       0
CaseNumber.1     0
originalorder    0
dtype: int64


In [276]:
#for index, row in df.iterrows():
    #if row["CaseNumber"] != row["CaseNumber.1"]:
        #print(row["CaseNumber"], " - ", row["CaseNumber.1"])

#TODO Maybe we can drop both columns as I don't think they hold relevant information and have some mismatched information (typos/incorrect format)

In [277]:
#Country , State , Location - We really only need one of these to identify the place, so lets drop all that have all 3 missing
print(df.shape)
df = df[df!= " "]
df = df[(df["State"].isna()==False) | (df["Country"].isna()==False) | (df["Location"].isna()==False)] #Removes all True|True|True which have na/na/na as the location
print(df.shape)


(6944, 20)
(6915, 20)


In [280]:
mini_df = df[df["Country"] == "USA"]

#print(mini_df.State.value_counts())

us_states_codes = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

#state_series = mini_df["State"].value_counts(dropna=False)

mini_df["State"] = mini_df["State"].str.strip()

mini_df["State"].replace(us_states_codes, inplace=True)

#disregard states with less that 10 occurences
state_counts = mini_df["State"].value_counts(dropna=False)

states_to_keep = state_counts[state_counts > 10].index
# Filter the DataFrame
USA_df = mini_df[mini_df["State"].isin(states_to_keep)]

print(USA_df["State"].value_counts(dropna=False))

df["State"].replace(to_replace={} , inplace=True)


State
FL             1179
HI              341
CA              323
SC              172
NC              121
TX               78
NJ               57
NY               48
OR               31
VA               19
LA               18
MA               18
GA               17
AL               17
Puerto Rico      17
Name: count, dtype: int64


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_47152\4039681287.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mini_df["State"] = mini_df["State"].str.strip()
C:\Users\LENOVO\AppData\Local\Temp\ipykernel_47152\4039681287.py:62: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  mini_df["S

In [297]:
#Florida Dataframe

FL_df = mini_df[mini_df["State"] == "FL"]
print(FL_df.shape)

print(FL_df.Type.value_counts())
print(FL_df.Year.value_counts())

years = FL_df.Year.value_counts()

for i in range(2000,2024):
    print(f"{i} -> {years[i]}")
    
FL_df.head(0)


(1179, 20)
Type
Unprovoked             982
Provoked               101
Invalid                 68
Watercraft              13
Sea Disaster             6
Questionable             4
Unconfirmed              1
Under investigation      1
Name: count, dtype: int64
Year
2003.0    45
2007.0    39
2001.0    38
2008.0    37
2000.0    35
          ..
1925.0     1
1954.0     1
1933.0     1
1934.0     1
1980.0     1
Name: count, Length: 116, dtype: int64
2000 -> 35
2001 -> 38
2002 -> 31
2003 -> 45
2004 -> 12
2005 -> 24
2006 -> 28
2007 -> 39
2008 -> 37
2009 -> 20
2010 -> 18
2011 -> 16
2012 -> 32
2013 -> 27
2014 -> 33
2015 -> 33
2016 -> 34
2017 -> 34
2018 -> 17
2019 -> 26
2020 -> 23
2021 -> 30
2022 -> 24
2023 -> 23
<bound method IndexOpsMixin.value_counts of 1       2024.0
2       2024.0
10      2023.0
24      2023.0
29      2023.0
         ...  
6891       0.0
6925       0.0
6930       0.0
6934       0.0
6941       0.0
Name: Year, Length: 2456, dtype: float64>


In [301]:

us = USA_df.Year.value_counts()
florida = FL_df.Year.value_counts()

i = 2000

for i in range(2000,2024):
    print(florida[i]/us[i]*100)



68.62745098039215
65.51724137931035
64.58333333333334
80.35714285714286
34.285714285714285
47.05882352941176
53.84615384615385
60.0
62.71186440677966
48.78048780487805
47.368421052631575
33.33333333333333
50.0
45.0
53.2258064516129
45.20547945205479
52.307692307692314
52.307692307692314
40.476190476190474
46.42857142857143
53.48837209302325
58.82352941176471
48.97959183673469
51.11111111111111
